In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.fullDataExtraction import extarctChannelNames
from ipynb.fs.full.feature_selection import get_features
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math
from tensorflow import keras
import dash_bootstrap_components as dbc

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [6]:
def makeEpochs(rawArrays):
    sliced_epochs={}
    for clip in rawArrays:
        sliced_epochs[clip]= mne.make_fixed_length_epochs(rawArrays[clip], duration=1, preload=True,verbose=0)
    return sliced_epochs

def getRawArrays(matfile):
    del matfile["__header__"]
    del matfile["__version__"]
    del matfile["__globals__"]
    clip_info={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    keyName=list(matfile.keys())[0][:-1]
    channelNamesExtraction=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    info=mne.create_info(channelNamesExtraction,200,'eeg')
    for ind,i in enumerate(indexs):
        rawData=matfile[keyName + str(i)]
        clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)
    return clip_info

def makeValueMatrix(currentAnalysed):
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    #tracker value
    k=0
    for i in range(0,result.shape[0]):
        for j in range (0,result.shape[1]):
            if not(math.isnan(result[i][j])):
                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result

def makePostionList(data):
    position=[]
    for column in range (0,500):
        position.append(makeValueMatrix(data[:,column]))
    return position

def ExtractFeatures(sliced_epochs,features):
    combined = np.zeros((1,len(features)*62))
    for cut in sliced_epochs:
        epoch_array=mne.Epochs.get_data(sliced_epochs[cut])
        extracted_data=extract_features(epoch_array,200,features)
        combined = np.vstack((combined,extracted_data))
    combined = np.delete(combined, 0, axis=0)
    return combined

In [3]:
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")
channelNames=channels.iloc[:,0]
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
channelNames.insert(0,"Fp1")
ch_types = ['eeg'] * len(channelNames)

In [4]:
info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)
info.set_montage('standard_1020')

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF3, AF4, F7, F5, F3, F1, Fz, F2, F4, F6, F8, ...
 chs: 60 EEG
 custom_ref_applied: False
 dig: 63 items (3 Cardinal, 60 EEG)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 60
 projs: []
 sfreq: 200.0 Hz
>

In [7]:
data=np.delete(subject['djc_eeg1'], 61, 0)
data=np.delete(data, 57, 0)
evoked=mne.EvokedArray(data,info)
transposedData=np.transpose(data)
df = pd.DataFrame(transposedData, columns = channelNames)
df = df.head(500)

In [9]:
model = keras.models.load_model("emotionPredictionModel")
subject=loadmat('1_20131107.mat')
rawArrays=getRawArrays(subject)
slicedEpochs=makeEpochs(rawArrays)
extractedData = ExtractFeatures(slicedEpochs,get_features())
points=makePostionList(data)

############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
Correlation Limit = 0.7
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Multi_Classification problem ####
    Loaded train data. Shape = (210429, 13)
#### Single_Label Multi_Classification problem ####
No test data filename given...
Classifying features using a random sample of 10000 rows from dataset...
#### Single_Label Multi_Classification problem ####
    loading a random sample of 10000 rows into pandas for EDA
#######################################################################################
##############

In [10]:
y_pred = model.predict(extractedData)
y_pred = (y_pred > 0.5).astype(int)
y_pred = y_pred.tolist()

107/107 [==============================] - 0s 584us/step


In [107]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])
slider = html.Div(
    [
            dcc.Slider(
            min=0,
            max=500,
            step=1,
            value=7,
            id="time",
            marks=None,
            tooltip={"always_visible": True},
            vertical=True
        ),
    ],
    style = {"height":"100%","text-align": "center",'padding-left':'50%', 'padding-right':'50%'}
)
app.layout=html.Div(children=[
                dbc.Row([
                    dbc.Col(slider,width=2),
                    dbc.Col([
                            dbc.Row([  
                                    html.H1(id="emotion")
                                    ], justify="center"),
                            dbc.Row([
                                    dbc.Col(dcc.Graph(id="lineChartFrontal"),width=4),
                                    dbc.Col(dcc.Graph(id="lineChartCentral"),width=4),
                                    dbc.Col(dcc.Graph(id="lineChartRightTemporal"),width=4)
                                    ]),
                            dbc.Row([
                                    dbc.Col(dcc.Graph(id="lineChartParietal"),width=4),
                                    dbc.Col(dcc.Graph(id="lineChartOccipital"),width=4),
                                    dbc.Col(dcc.Graph(id="lineChartLeftTemporal"),width=4)
                                    ]),
                            dbc.Row([
                                    dbc.Col(dcc.Graph(id="heatmap"),width=4)
                                    ]),
                                
                        ],width=10)
                    ])
                ])
#function for updating heatmap when slider is moved
@app.callback(Output("heatmap", "figure"),
              Input("time", "value"))
def updateHeatMap(time_instance):
    heatMap = px.imshow(points[time_instance])
    heatMap.update_xaxes(showticklabels=False)
    heatMap.update_yaxes(showticklabels=False)
    heatMap.update_layout(xaxis=dict(scaleanchor='y', constrain='domain'))
    return heatMap
#function for updating emotion prediction
@app.callback(Output("emotion", "children"),
              Input("time", "value"))
def updatePrediction(time_instance):
    if (y_pred[time_instance] == [1,0,0]):
        emotion="positive"
    elif (y_pred[time_instance] == [0,1,0]):
        emotion="nuetral"
    else:
        emotion="negative"  
    emotionDiv=html.H1(emotion)
    return emotionDiv
#function for updating the line charts
@app.callback(Output("lineChartFrontal", "figure"),
              Output("lineChartCentral", "figure"),
              Output("lineChartParietal", "figure"),
              Output("lineChartOccipital", "figure"),
              Output("lineChartLeftTemporal", "figure"),
              Output("lineChartRightTemporal", "figure"),
              Input("time", "value"))
def updateLineCharts(time_instance):
    lineChartFrontal= px.line(df[['Fp1', 'Fpz', 'Fp2','AF3', 'AF4','F1', 'Fz', 'F2',]].head(time_instance))
    lineChartCentral= px.line(df[['FC1', 'FCz', 'FC2','C3', 'C1', 'Cz', 'C2', 'C4']].head(time_instance))
    lineChartParietal= px.line(df[['CP3', 'CP1', 'CPz', 'CP2', 'CP4','P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6']].head(time_instance))
    lineChartOccipital= px.line(df[['PO7', 'PO5', 'PO3','POz', 'PO4', 'PO6', 'PO8','O1', 'Oz', 'O2']].head(time_instance))
    lineChartLeftTemporal= px.line(df[['F4', 'F6', 'F8','FC4','FC6', 'FT8','C6','CP6', 'TP8','P8']].head(time_instance))
    lineChartFrontal= px.line(df[['F7', 'F5', 'F3','FT7', 'FC5', 'FC3','C5','TP7', 'CP5','P7']].head(time_instance))
    return lineChartFrontal,lineChartCentral,lineChartParietal,lineChartOccipital,lineChartLeftTemporal,lineChartFrontal
if __name__ == "__main__":
    app.run_server()

Dash app running on http://127.0.0.1:8050/


In [96]:
df.columns

Index(['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2',
       'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4',
       'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
       'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P7',
       'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3',
       'POz', 'PO4', 'PO6', 'PO8', 'O1', 'Oz', 'O2'],
      dtype='object')